In [8]:
ISTA_lassosolve <- function(X, y, lambda, max_iter = 1000, tol = 1e-6) {
  n <- nrow(X)
  p <- ncol(X)

  # Center X and y for numerical stability
  X_mean <- colMeans(X)
  y_mean <- mean(y)
  X_centered <- scale(X, center = X_mean, scale = FALSE)
  y_centered <- y - y_mean

  # Precompute X'X and X'y
  XtX <- crossprod(X_centered) / n
  Xty <- crossprod(X_centered, y_centered) / n

  # Lipschitz constant (L = max eigenvalue of XtX)
  L <- max(eigen(XtX, symmetric = TRUE, only.values = TRUE)$values)

  # Initialize beta (use Ridge regression initialization for better accuracy)
  beta <- solve(XtX + diag(lambda, p), Xty)

  # Soft-thresholding function
  soft_threshold <- function(z, gamma) {
    sign(z) * pmax(0, abs(z) - gamma)
  }

  # ISTA Iterations
  for (iter in 1:max_iter) {
    # Gradient calculation
    gradient <- XtX %*% beta - Xty
    beta_new <- soft_threshold(beta - (1 / L) * gradient, lambda / L)

    # Check convergence using relative error
    if (sqrt(sum((beta_new - beta)^2)) / max(1, sqrt(sum(beta^2))) < tol) {
      intercept <- y_mean - sum(X_mean * beta_new)  # Restore intercept
      full_beta <- c(intercept, beta_new)          # Combine intercept and coefficients
      return(list(beta = full_beta, iter = iter, convergence = TRUE))
    }

    # Update beta
    beta <- beta_new
  }

  # Final intercept calculation
  intercept <- y_mean - sum(X_mean * beta)
  full_beta <- c(intercept, beta)

  return(list(beta = full_beta, iter = max_iter, convergence = FALSE))
}


In [9]:
set.seed(123)

# 模拟数据
n <- 100  # 样本数
p <- 10   # 特征数
X <- matrix(rnorm(n * p), n, p)           # 特征矩阵
beta_true <- c(5, 1, -1, rep(0, p - 2))  # 包括截距项（5）和稀疏系数
y <- X %*% beta_true[-1] + beta_true[1] + rnorm(n)  # 生成响应变量

# 正则化参数
lambda <- 0.1

# 调用 ISTA_lassosolve
result <- ISTA_lassosolve(X, y, lambda)

# 查看结果
cat("Fitted coefficients (with intercept):\n")
print(result$beta)  # 包括截距项的系数
cat("Number of iterations:", result$iter, "\n")
cat("Convergence:", result$convergence, "\n")


Fitted coefficients (with intercept):
 [1]  5.14042631  0.95608943 -0.91324410 -0.06293648  0.07228975  0.00000000
 [7]  0.00000000  0.00000000  0.04065107  0.00000000  0.06717730
Number of iterations: 19 
Convergence: TRUE 


In [2]:
ISTA_solver <- function(X, y, lambda, max_iter=1000, tol=1e-6) {
  x <- rep(0, ncol(X))
  beta <- numeric(max_iter)
  L <- norm(X, "2")^2
  start_time <- Sys.time()
  for (iter in 1 : max_iter) {
    x_new <- soft_thresh(x + t(X) %*% (y - X %*% x) / L, lambda / L)
    print(x_new)
    beta[iter] <- 0.5 * norm(X %*% x_new - y, "F")^2 + lambda * sum(abs(x_new))
    
    if (sqrt(sum((x_new - x)^2)) < tol) {
      return(list(x = x_new, beta = beta[1 : iter], 
                  time = Sys.time() - start_time, 
                  convergence = TRUE, iter = iter))
    }
    x <- x_new
  }
  return(list(x = x, beta = beta, time = Sys.time() - start_time, 
              convergence = FALSE, iter = max_iter))
}

soft_thresh <- function(x, lambda) {
  sign(x) * pmax(abs(x) - lambda, 0)
}

In [3]:
ISTA_lassosolve <- function(X, y, lambda, max_iter = 1000, tol = 1e-6) {
  n <- nrow(X)
  p <- ncol(X)

  # Center X and y for numerical stability
  X_mean <- colMeans(X)
  y_mean <- mean(y)
  X_centered <- scale(X, center = X_mean, scale = FALSE)
  y_centered <- y - y_mean

  # Precompute X'X and X'y
  XtX <- crossprod(X_centered) / n
  Xty <- crossprod(X_centered, y_centered) / n

  # Lipschitz constant (L = max eigenvalue of XtX)
  L <- max(eigen(XtX, symmetric = TRUE, only.values = TRUE)$values)

  # Initialize beta (use Ridge regression initialization for better accuracy)
  beta <- solve(XtX + diag(lambda, p), Xty)

  # Soft-thresholding function
  soft_threshold <- function(z, gamma) {
    sign(z) * pmax(0, abs(z) - gamma)
  }

  # ISTA Iterations
  for (iter in 1:max_iter) {
    # Gradient calculation
    gradient <- XtX %*% beta - Xty
    beta_new <- soft_threshold(beta - (1 / L) * gradient, lambda / L)

    # Check convergence using relative error
    if (sqrt(sum((beta_new - beta)^2)) / max(1, sqrt(sum(beta^2))) < tol) {
      intercept <- y_mean - sum(X_mean * beta_new)  # Restore intercept
      full_beta <- c(intercept, beta_new)          # Combine intercept and coefficients
      return(list(beta = full_beta, iter = iter, convergence = TRUE))
    }

    # Update beta
    beta <- beta_new
  }

  # Final intercept calculation
  intercept <- y_mean - sum(X_mean * beta)
  full_beta <- c(intercept, beta)

  return(list(beta = full_beta, iter = max_iter, convergence = FALSE))
}


In [4]:
set.seed(123)

# 模拟数据
n <- 100  # 样本数
p <- 10   # 特征数
X <- matrix(rnorm(n * p), n, p)           # 特征矩阵
beta_true <- c(5, 1, -1, rep(0, p - 2))  # 包括截距项（5）和稀疏系数
y <- X %*% beta_true[-1] + beta_true[1] + rnorm(n)  # 生成响应变量
# 正则化参数
lambda <- 0.1

# 调用 ISTA_lassosolve
lassosolve_result <- ISTA_lassosolve(X, y, lambda)
lassosolve_beta <- lassosolve_result$beta
y_pred_lassosolve <- cbind(1, X) %*% lassosolve_beta  # Include intercept
residuals_lassosolve <- y - y_pred_lassosolve
mse_lassosolve <- mean(residuals_lassosolve^2)
r2_lassosolve <- 1 - sum(residuals_lassosolve^2) / sum((y - mean(y))^2)
print("lassosolve", mse_lassosolve, r2_lassosolve)

result_solver <- ISTA_solver(X, y, lambda)
solver_beta <- result_solver$beta
y_pred_solver <- cbind(1, X) %*% solver_beta  # Include intercept
residuals_rsolve <- y - y_pred_solver
residuals_solver <- y - y_pred_solver
mse_solver <- mean(residuals_solver^2)
r2_solver <- 1 - sum(residuals_solver^2) / sum((y - mean(y))^2)
print("solver", mse_solver, r2_solver)

glmnet_result <- glmnet(X, y, alpha = 1, lambda = lambda, intercept = TRUE)

glmnet_beta <- as.vector(coef(glmnet_result, s = lambda))
y_pred_glmnet <- cbind(1, X) %*% glmnet_beta  # Include intercept
residuals_glmnet <- y - y_pred_glmnet

# Calculate metrics for glmnet
mse_glmnet <- mean(residuals_glmnet^2)
r2_glmnet <- 1 - sum(residuals_glmnet^2) / sum((y - mean(y))^2)
print("glmnet", mse_glmnet, r2_glmnet)

[1] "lassosolve"
             [,1]
 [1,]  0.96109459
 [2,] -1.05875485
 [3,]  0.23255960
 [4,] -0.04512378
 [5,]  0.37126105
 [6,] -0.28084212
 [7,] -0.55081928
 [8,]  0.61748196
 [9,]  0.28895390
[10,]  0.12050415
             [,1]
 [1,]  1.29910325
 [2,] -1.28671399
 [3,]  0.44729611
 [4,] -0.07710151
 [5,]  0.59399704
 [6,] -0.40306661
 [7,] -0.57419805
 [8,]  0.72190583
 [9,]  0.29386605
[10,]  0.08814526
             [,1]
 [1,]  1.45920742
 [2,] -1.32691699
 [3,]  0.56310312
 [4,] -0.07166961
 [5,]  0.71003714
 [6,] -0.46347806
 [7,] -0.55585007
 [8,]  0.75148948
 [9,]  0.29830126
[10,]  0.06992842
             [,1]
 [1,]  1.54342589
 [2,] -1.32338313
 [3,]  0.62239092
 [4,] -0.06082338
 [5,]  0.77421919
 [6,] -0.49619915
 [7,] -0.54231028
 [8,]  0.76130246
 [9,]  0.30985865
[10,]  0.06179200
             [,1]
 [1,]  1.58993220
 [2,] -1.31237165
 [3,]  0.65291568
 [4,] -0.05205536
 [5,]  0.81130015
 [6,] -0.51544709
 [7,] -0.53442439
 [8,]  0.76467489
 [9,]  0.32070689
[10,]  0.05

ERROR: Error in cbind(1, X) %*% solver_beta: non-conformable arguments
